In [13]:
import kfp
import os
import requests

from kfp.dsl import Input, Model, component, Dataset, Output, Artifact
from kfp.dsl import InputPath, OutputPath, pipeline, component, PipelineTask
from kfp.components import load_component_from_file

In [14]:
@component(
    base_image='python:3.9',
    packages_to_install=["gitpython"],
)
def download_files_from_github(repo_url: str, target_subdir: str, output_dir: Output[Artifact]):
    import git
    import shutil
    from pathlib import Path

    # repo_url = "https://github.com/ravikiranvs/kf-demo.git"
    # target_subdir = "examples/kube-ray/job"

    # Clone the repo to /tmp
    repo_path = "/tmp/kf-demo"
    if Path(repo_path).exists():
        shutil.rmtree(repo_path)
    git.Repo.clone_from(repo_url, repo_path)

    # Copy the desired files to the output directory
    source_dir = Path(repo_path) / target_subdir
    dest_dir = Path(output_dir.path)
    dest_dir.mkdir(parents=True, exist_ok=True)

    for file_path in source_dir.iterdir():
        if file_path.is_file():
            shutil.copy(file_path, dest_dir / file_path.name)

In [15]:
@component(
    base_image="rayproject/ray:2.41.0"
)
def submit_job(job_dir: Input[Artifact], host: str, port: int):
    import os
    import shutil
    from ray.job_submission import JobSubmissionClient

    target_dir = "./code"
    if os.path.exists(target_dir):
        shutil.rmtree(target_dir)
    shutil.copytree(job_dir.path, target_dir)

    client = JobSubmissionClient(f"http://{host}:{port}")

    job_id = client.submit_job(
        entrypoint="python ray-job.py",
        runtime_env={
            "working_dir": target_dir,
            "pip": [
                "accelerate",
                "transformers[torch]",
                "transformers",
                "datasets",
                "evaluate",
                "huggingface_hub",
                "peft",
                "bitsandbytes",
                "deepspeed",
                "ray[train]"
            ]
        }
    )
    
    print(job_id)

In [16]:
@pipeline(name='ray-job-submit-pipeline')
def file_processing_pipeline(host: str, port: int):
    job_download_op = download_files_from_github(repo_url="https://github.com/ravikiranvs/kf-demo.git", target_subdir="examples/kube-ray/job")
    job_op = submit_job(job_dir=job_download_op.output, host=host, port=port)

In [18]:
client = kfp.Client()
run = client.create_run_from_pipeline_func(file_processing_pipeline, arguments={"host": "raycluster-kuberay-head-svc.demo-ns.svc.cluster.local", "port": 8265}, enable_caching=False)